In [1]:
import torch
import timm
from tqdm import tqdm
from aimet_torch.batch_norm_fold import fold_all_batch_norms
from aimet_torch.model_preparer import prepare_model
from aimet_torch.model_validator.model_validator import ModelValidator
import aimet_torch.quantsim as qsim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torch.utils.data import Subset

/home/bmw/anaconda3/envs/swin_bhanu/lib/python3.10/site-packages/onnxscript/converter.py:823: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/home/bmw/anaconda3/envs/swin_bhanu/lib/python3.10/site-packages/onnxscript/converter.py:823: FutureWarning: 'onnxscript.values.OnnxFunction.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()


2025-03-02 17:18:22,544 - root - INFO - AIMET


/home/bmw/anaconda3/envs/swin_bhanu/lib/python3.10/site-packages/torch/overrides.py:110: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  torch.has_cuda,
/home/bmw/anaconda3/envs/swin_bhanu/lib/python3.10/site-packages/torch/overrides.py:111: UserWarning: 'has_cudnn' is deprecated, please use 'torch.backends.cudnn.is_available()'
  torch.has_cudnn,
/home/bmw/anaconda3/envs/swin_bhanu/lib/python3.10/site-packages/torch/overrides.py:117: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  torch.has_mps,
/home/bmw/anaconda3/envs/swin_bhanu/lib/python3.10/site-packages/torch/overrides.py:118: UserWarning: 'has_mkldnn' is deprecated, please use 'torch.backends.mkldnn.is_available()'
  torch.has_mkldnn,


In [2]:
model = timm.create_model("swin_tiny_patch4_window7_224", pretrained=True)
model.eval()

/home/bmw/anaconda3/envs/swin_bhanu/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


2025-03-02 17:18:26,369 - timm.models.helpers - INFO - Loading pretrained weights from url (https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_tiny_patch4_window7_224.pth)


SwinTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (layers): Sequential(
    (0): BasicLayer(
      dim=96, input_resolution=(56, 56), depth=2
      (blocks): ModuleList(
        (0): SwinTransformerBlock(
          (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (attn): WindowAttention(
            (qkv): Linear(in_features=96, out_features=288, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=96, out_features=96, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (softmax): Softmax(dim=-1)
          )
          (drop_path): Identity()
          (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=96, out_features=384, bias=True)
            (a

In [3]:
def validate_pytorch(model, dataloader, device, subset=False):
    model.eval()
    top1_correct = 0
    top5_correct = 0
    total = 0

    with torch.no_grad():
        for i, (images, labels) in enumerate(tqdm(dataloader, desc="Validating PyTorch Model", leave=False)):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, pred_top1 = outputs.topk(1, dim=1)
            _, pred_top5 = outputs.topk(5, dim=1)

            top1_correct += (pred_top1.squeeze() == labels).sum().item()
            top5_correct += sum([labels[i] in pred_top5[i] for i in range(len(labels))])
            total += labels.size(0)

            if subset and total >= 100:  # Validate only the first 100 samples for subset check
                break

    top1_acc = 100 * top1_correct / total
    top5_acc = 100 * top5_correct / total
    return top1_acc, top5_acc

In [4]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
dataset = datasets.ImageFolder("/media/bmw/datasets/imagenet-1k/val", transform=transform)

subset_size = 1000
subset_indices = list(range(subset_size))  # Taking first 1000 samples
subset_dataset = Subset(dataset, subset_indices)

# Create DataLoader for the subset
BATCH_SIZE = 32
subset_loader = DataLoader(subset_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

# Verify DataLoader
for images, labels in subset_loader:
    print(images.shape, labels.shape)  # Example output: torch.Size([32, 3, 224, 224]) torch.Size([32])
    break  # Print only the first batch

dataloader = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

device = torch.device("cpu")

torch.Size([32, 3, 224, 224]) torch.Size([32])


In [5]:
print("\nPreparing and validating the model for quantization...")
prepared_model = prepare_model(model).to(device)


Preparing and validating the model for quantization...
2025-03-02 17:18:37,031 - ModelPreparer - INFO - Functional         : Adding new module for node: {layers.0.blocks.0.module_floordiv} 
2025-03-02 17:18:37,031 - ModelPreparer - INFO - Functional         : Adding new module for node: {layers.0.blocks.0.module_floordiv_1} 
2025-03-02 17:18:37,032 - ModelPreparer - INFO - Functional         : Adding new module for node: {layers.0.blocks.0.attn.module_floordiv_2} 
2025-03-02 17:18:37,032 - ModelPreparer - INFO - Functional         : Adding new module for node: {layers.0.blocks.0.attn.module_mul} 
2025-03-02 17:18:37,032 - ModelPreparer - INFO - Functional         : Adding new module for node: {layers.0.blocks.0.attn.module_matmul} 
2025-03-02 17:18:37,033 - ModelPreparer - INFO - Functional         : Adding new module for node: {layers.0.blocks.0.attn.module_add} 
2025-03-02 17:18:37,033 - ModelPreparer - INFO - Functional         : Adding new module for node: {layers.0.blocks.0.attn.

In [6]:
validation_result = ModelValidator.validate_model(prepared_model, model_input=torch.rand(1, 3, 224, 224).to(device))

2025-03-02 17:18:39,088 - Utils - INFO - Running validator check <function validate_for_reused_modules at 0x7f1b19f9f2e0>
2025-03-02 17:18:39,147 - Utils - INFO - Running validator check <function validate_for_missing_modules at 0x7f1b19f9f5b0>
2025-03-02 17:18:48,986 - ConnectedGraph - WARNING - Unable to isolate model outputs.
2025-03-02 17:18:49,011 - Utils - ERROR - Functional ops that are not marked as math invariant were found in the model. AIMET features will not work properly for such ops.
Consider the following choices: 
1. Redefine as a torch.nn.Module in the class definition.
2. The op can remain as a functional op due to being math invariant, but the op type has not been added to ConnectedGraph.math_invariant_types set. 
Add an entry to ignore the op by importing the set and adding the op type:

	from aimet_torch.meta.connectedgraph import ConnectedGraph
	ConnectedGraph.math_invariant_types.add(...)

The following functional ops were found. The parent module is named for ea

In [7]:
from aimet_torch.meta.connectedgraph import ConnectedGraph

ConnectedGraph.math_invariant_types.add("unbind")
ConnectedGraph.math_invariant_types.add("roll")
ConnectedGraph.math_invariant_types.add("remainder")
ConnectedGraph.math_invariant_types.add("floor-divide")

In [8]:
validation_result = ModelValidator.validate_model(prepared_model, model_input=torch.rand(1, 3, 224, 224).to(device))

2025-03-02 17:18:54,266 - Utils - INFO - Running validator check <function validate_for_reused_modules at 0x7f1b19f9f2e0>
2025-03-02 17:18:54,317 - Utils - INFO - Running validator check <function validate_for_missing_modules at 0x7f1b19f9f5b0>
2025-03-02 17:19:03,859 - ConnectedGraph - WARNING - Unable to isolate model outputs.
2025-03-02 17:19:03,885 - Utils - INFO - All validation checks passed.


In [9]:
def apply_aimet_quantization(model, device, dataloader):
    dummy_input = torch.rand(1, 3, 224, 224).to(device)

    # Fold batch normalization layers
    fold_all_batch_norms(model, input_shapes=(1, 3, 224, 224), dummy_input=dummy_input)

    quant_sim = qsim.QuantizationSimModel(
        model,
        dummy_input=dummy_input,
        quant_scheme='tf_enhanced',
        default_param_bw=8,
        default_output_bw=8
    )

    # Calibration using a small dataset
    def forward_pass(model, dataloader):
        model.eval()
        with torch.no_grad():
            for i, (images, _) in enumerate(dataloader):
                if i >= 10:  # Use only first 10 batches for calibration
                    break
                model(images.to(device))

    quant_sim.compute_encodings(forward_pass, dataloader)
    quant_sim.export(path="/media/bmw/Akshaya/AIMET/swin/artifacts",filename_prefix="swin_quantized",dummy_input=dummy_input)
    
    return quant_sim.model

In [10]:
quantized_model = apply_aimet_quantization(prepared_model, device, dataloader)
print("\nValidating quantized model...")
top1_acc, top5_acc = validate_pytorch(quantized_model, dataloader, device)
print(f"Quantized Model - Top-1 Accuracy: {top1_acc:.2f}%, Top-5 Accuracy: {top5_acc:.2f}%")

2025-03-02 17:19:19,721 - ConnectedGraph - WARNING - Unable to isolate model outputs.
2025-03-02 17:19:30,978 - ConnectedGraph - WARNING - Unable to isolate model outputs.
2025-03-02 17:19:31,149 - Quant - INFO - No config file provided, defaulting to config file at /home/bmw/anaconda3/envs/swin_bhanu/lib/python3.10/site-packages/aimet_common/quantsim_config/default_config.json
2025-03-02 17:19:31,172 - Quant - INFO - Unsupported op type Squeeze
2025-03-02 17:19:31,172 - Quant - INFO - Unsupported op type Mean
2025-03-02 17:19:31,195 - Quant - INFO - Selecting DefaultOpInstanceConfigGenerator to compute the specialized config. hw_version:default
2025-03-02 17:19:31,199 - Quant - WARNING - Multiple input ops exist, traversal to find closest producer is performed based on the first input
2025-03-02 17:19:31,199 - Quant - WARNING - Multiple input ops exist, traversal to find closest producer is performed based on the first input
2025-03-02 17:19:31,200 - Quant - WARNING - Multiple input o

Quantized Model - Top-1 Accuracy: 78.17%, Top-5 Accuracy: 94.12%


AUTOQUANT

In [ ]:
import torch
import numpy as np

from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Subset

# Set device
device = torch.device("cpu")

In [ ]:

# Define batch size
BATCH_SIZE = 64
SUBSET_SIZE = 1000  # Define how many samples to take

# Define transformation function
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Set dataset paths
train_data_path = "/media/bmw/datasets/imagenet-1k/train/"
val_data_path = "/media/bmw/datasets/imagenet-1k/val/"

# Load dataset using ImageFolder
train_dataset = datasets.ImageFolder(root=train_data_path, transform=preprocess)
val_dataset = datasets.ImageFolder(root=val_data_path, transform=preprocess)

# Take a subset of the validation dataset
subset_indices = list(range(SUBSET_SIZE))  # First `SUBSET_SIZE` samples
val_subset = Subset(val_dataset, subset_indices)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

from typing import Optional
# Define evaluation function
def eval_callback(model: torch.nn.Module,num_samples: Optional[int] = None) -> float:
    acc_top1 = 0
    total_samples = 0
    
    dataloader = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

    model.to(device)
    model.eval()

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)

            pred_probs = model(images)
            pred_labels = pred_probs.argmax(dim=1)

            acc_top1 += (pred_labels == labels).sum().item()
            total_samples += labels.size(0)

    return acc_top1 / total_samples

In [ ]:
accuracy = eval_callback(model)
print(f"Validation Accuracy on Subset: {accuracy:.4f}")

Validation Accuracy on Subset: 0.9310


In [ ]:
from aimet_torch.model_validator.model_validator import ModelValidator

ModelValidator.validate_model(prepared_model,torch.randn(1,3,224,224))

2025-03-01 17:21:34,408 - Utils - INFO - Running validator check <function validate_for_reused_modules at 0x7fee366972e0>
2025-03-01 17:21:34,461 - Utils - INFO - Running validator check <function validate_for_missing_modules at 0x7fee366975b0>
2025-03-01 17:21:44,257 - ConnectedGraph - WARNING - Unable to isolate model outputs.
2025-03-01 17:21:44,282 - Utils - INFO - All validation checks passed.


True

In [36]:
from torchvision.datasets import ImageFolder

def load_unlabelled_data(dataset, batch_size) -> DataLoader :
    class UnlabelledDataset(ImageFolder):
        def __init__(self, dataset):
            self._dataset = dataset

        def __len__(self):
            return len(self._dataset)

        def __getitem__(self, index):
            images, _ = self._dataset[index]
            return images
        
    unlabelled_dataset=UnlabelledDataset(dataset)
    
    unlabelled_dataloader= DataLoader (
    unlabelled_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=8,
    pin_memory= True
    )
    
    return unlabelled_dataset,unlabelled_dataloader


In [37]:
unlabelled_val_dataset, unlabelled_val_dataloader  = load_unlabelled_data(val_subset, 64)

In [38]:
from aimet_torch.meta.connectedgraph import ConnectedGraph
ConnectedGraph.math_invariant_types.add("floor_divide")

In [39]:
from aimet_torch.auto_quant import AutoQuant
dummy_input = torch.randn(1,3,224,224)

auto_quant= AutoQuant(prepared_model,dummy_input=dummy_input,data_loader=unlabelled_val_dataloader,eval_callback=eval_callback)

sim, initial_accuracy = auto_quant.run_inference()
print(f"Quantization accuracy before optmization {initial_accuracy}")

2025-03-01 17:24:24,045 - ModelPreparer - INFO - Functional         : Adding new module for node: {module_mul} 


2025-03-01 17:24:24,046 - ModelPreparer - INFO - Functional         : Adding new module for node: {module_matmul} 


2025-03-01 17:24:24,048 - ModelPreparer - INFO - Functional         : Adding new module for node: {module_add} 


2025-03-01 17:24:24,049 - ModelPreparer - INFO - Functional         : Adding new module for node: {module_matmul_1} 


2025-03-01 17:24:24,050 - ModelPreparer - INFO - Functional         : Adding new module for node: {module_add_1} 


...

2025-03-01 17:36:09,186 - AutoQuant - INFO - Session finished: Batchnorm Folding. (eval score: 0.913000)


\ Batchnorm Folding


2025-03-01 17:36:20,529 - ConnectedGraph - WARNING - Unable to isolate model outputs.
2025-03-01 17:36:20,694 - Quant - INFO - No config file provided, defaulting to config file at /home/bmw/anaconda3/envs/swin_bhanu/lib/python3.10/site-packages/aimet_common/quantsim_config/default_config.json
2025-03-01 17:36:20,715 - Quant - INFO - Unsupported op type Squeeze
2025-03-01 17:36:20,716 - Quant - INFO - Unsupported op type Mean
2025-03-01 17:36:20,736 - Quant - INFO - Selecting DefaultOpInstanceConfigGenerator to compute the specialized config. hw_version:default
2025-03-01 17:36:20,740 - Quant - WARNING - Multiple input ops exist, traversal to find closest producer is performed based on the first input
2025-03-01 17:36:20,740 - Quant - WARNING - Multiple input ops exist, traversal to find closest producer is performed based on the first input
2025-03-01 17:36:20,741 - Quant - WARNING - Multiple input ops exist, traversal to find closest producer is performed based on the first input
202

100%|██████████| 16/16 [09:37<00:00, 36.11s/it]


Quantization accuracy before optmization 0.913


In [40]:
from aimet_torch.meta.connectedgraph import ConnectedGraph
ConnectedGraph.math_invariant_types.add("floor_divide")

In [41]:
from aimet_torch.adaround.adaround_weight import AdaroundParameters

adaround_data_loader = val_loader

adaround_params = AdaroundParameters(adaround_data_loader, num_batches=len(adaround_data_loader), default_num_iterations=100)
auto_quant.set_adaround_params(adaround_params)

In [42]:
model, optimized_accuracy, encoding_path = auto_quant.optimize(allowed_accuracy_drop=0.01)
print(f"- Quantized Accuracy (after optimization):  {optimized_accuracy}")

2025-03-01 17:54:57,184 - AutoQuant - INFO - Starting AutoQuant
2025-03-01 17:55:29,930 - AutoQuant - INFO - Target eval score: 0.921000
2025-03-01 17:55:29,931 - AutoQuant - INFO - FP32 eval score (W32A32): 0.931000


- Prepare Model


2025-03-01 17:55:43,180 - ConnectedGraph - WARNING - Unable to isolate model outputs.


2025-03-01 17:55:43,364 - Quant - INFO - No config file provided, defaulting to config file at /home/bmw/anaconda3/envs/swin_bhanu/lib/python3.10/site-packages/aimet_common/quantsim_config/default_config.json


2025-03-01 17:55:43,386 - Quant - INFO - Unsupported op type Squeeze


...

2025-03-01 19:01:45,836 - Quant - WARNING - Quantsim export will stop exporting encodings for saving and loading in a future AIMET release.
To export encodings for saving and loading, use QuantizationSimModel's save_encodings_to_json() utility instead.


2025-03-01 19:01:45,842 - AutoQuant - INFO - The results of Cross-Layer Equalization is saved in /tmp/.trace/cross_layer_equalization.pth and /tmp/.trace/cross_layer_equalization.encodings.


2025-03-01 19:01:45,844 - AutoQuant - INFO - Session finished: Cross-Layer Equalization. (eval score: 0.935000)


- Cross-Layer Equalization


2025-03-01 19:01:46,154 - AutoQuant - INFO - Best eval score: 0.935000
- Quantized Accuracy (after optimization):  0.935
